# Take Home Activity
Author: Charlie Moffett, September 2018

As a Sales Engineering candidate with Samsara, I was gifted a kit of gear to outfit my vehicle with as if I were hauling temperature controlled cargo. In this noteboook, I'll be digging into the hardware setup, Samsara Dashboard administration, and API workings of the product. You'll learn how I used Samsara's integrated, software-centric solution to extract insights from  my Labor Day road trip to demonstrate how customers can leverage data from the physical world to increase operational efficiency.

### Hardware Setup

The first thing I did was set up all the Samsara hardware that I was provided (pictured below). Not pictured is the OBDII cable used to install my Samsara Vehicle IoT Gateway (VG) - the hub of Samsara's complete solution. It integrates information from a variety of built-in and accessory sensors, including the environmental and door monitors included in my kit. This type of comprehensive and intuitive view is ideal for my simulation of a refrigerated fleet.  

For more about the gear and setup, visit https://www.samsara.com/support.

| <img src='https://s3-us-west-2.amazonaws.com/corpweb-static/img/vg34-front-detail.jpg' width="200">(https://www.samsara.com/products/models/vg34)  | [![EM22](https://webobjects2.cdw.com/is/image/CDW/4462845)](https://www.samsara.com/products/models/em22) | [![DM11](https://webobjects2.cdw.com/is/image/CDW/4920488)](https://www.samsara.com/products/models/dm11) | 
|:---:|:---:|:---:|
| Vehicle Gateway (VG34) | Environmental Monitor (EM22) | Door Monitor (DM11) |

### Configuring the Dashboard

Next step was to create an account and register my hardware at samara.com/activate. My Samsara Dashboard (or Samsara Cloud) brings into view both real-time and historical sensor data over the web, allowing me to manage and monitor the equipment, create reports, configure alerts, etc. Though this activity probably falls on the smaller end of the scale, it's good to know that my data is stored indefinitely, with no quantity limits.  
  
Once I got the hardware up and running, I gave it the entirety of my traffic-ridden, Labor Day "Long Haul" to collect some road trip data. Here's an example of how one of the actionable data views could look in your dashboard:

<img src="https://s3-us-west-2.amazonaws.com/corpweb-static/img/screenshots/utilization-overview.jpg" alt="Samsara Cloud" width="500"/>

### Samsara Cloud API

The Samasara Cloud API allows access to all of your data in the Samsara Cloud via an HTTPS-based RPC-style API. In this exercise, I used the API endpoints to track and analyze my sensors and passenger vehicles. The interactions faciliated by the API enables Samsara users to build powerful applications and custom solutions with sensor data. Detailed information can be found in the API Guide here: https://www.samsara.com/api  

Now I'll **walk through the code** I wrote to extract data from Samsara Cloud in order to visualize my trip insights, starting with my package imports:

In [336]:
## This notebook was written in Python 3

# Python SDK for the Samsara REST API
import samsara
from samsara.rest import ApiException

## pretty printing
from pprint import pprint

# Create a dataframe to easily handle data for analysis and graphing
import pandas as pd
# Package for scientific computing
import numpy as np

# a graphing library for making interactive, publication-quality graphs
import plotly
# use Plotly's python API to plot inside the Jupyter Notebook
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff

# save graphs generated by plotly as an image
import plotly.offline as offline

# set plotly credentials
plotly.tools.set_credentials_file(username='cmoffett', api_key='<your_plotly_token')

In [337]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# initiate the Plotly Notebook mode to plot graphs offline inside a Jupyter Notebook Environment
init_notebook_mode(connected=True)
offline.init_notebook_mode()

print(__version__) # requires plotly version >= 1.9.0

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


2.4.1


### Accessing the API

Before I started developing with Samsara APIs, I had to obtain an API key from my Samsara Dashboard to authenticate my API requests. All API requests must be made over HTTPS (calls made over plain HTTP or without authentication will fail).  

The Samsara Cloud API is a RESTful API accessed by an HTTP client such as wget or **curl**. Here I'm using curl to perform a POST request to the **/fleets/list** endpoint URL. This method returns a list of the vehicles in my Samsara Cloud and information about them. 

In [207]:
# Get list of the vehicles
!curl -i -X POST -d "{\"groupId\": 18750}" "https://api.samsara.com/v1/fleet/list?access_token=<your_access_token>"
## All URIs are relative to https://api.samsara.com/v1

HTTP/1.1 200 OK
Date: Wed, 05 Sep 2018 17:20:50 GMT
Content-Type: application/json; charset=utf-8
Content-Length: 126
Connection: keep-alive

{"vehicles":[{"id":212014918214656,"name":"GCW9-EHX-PVV","vin":"4S4BRBGC9A3317040","odometerMeters":null,"engineHours":null}]}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   144  100   126  100    18    126     18  0:00:01 --:--:--  0:00:01   342


As we can see from the example above, JSON is returned by all API responses (incl. errors).  
We can see vehicle information in the JSON object here, including vehicle ID and VIN... **Hey, that's my car!**

### Python SDK  
If you prefer to interact with the API using Javascript or Python (like me), Samsara has SDKs available for download from https://github.com/samsarahq

Let's now dive into extracting sensor data from my road trip, which we'll munge and manage to prep for visualization.

In [209]:
## GET SENSOR OBJECTS
## This method returns a list of the sensor objects in the Samsara Cloud and information about them.

# create an instance of the Default API class
api_instance = samsara.DefaultApi()

## Many Samsara API endpoints require your Group ID as query or body parameter for functioning correctly
## Group ID can be viewed on the API Tokens Settings page 

# GroupParam | Group ID to query
group_param = samsara.GroupParam(18750)

In [210]:
## The get_sensors method is comparable to a POST request to the /sensors/list endpoint
try: 
    # /sensors/list
    api_response = api_instance.get_sensors(access_token, group_param) # use access token from curl example
    pprint(api_response)
except ApiException as e:
    print("Exception when calling SensorsApi->get_sensors: %s\n" % e)

{'sensors': [{'id': 212014918118242,
              'mac_address': 'c0:d3:91:e1:ab:62',
              'name': 'Environmental'},
             {'id': 212014918223924,
              'mac_address': 'c0:d3:91:e3:48:34',
              'name': 'Door'}]}


As we can see from the sensors list response object, I have two monitors feeding into Samsara Cloud via the vehicle gateway: **Environmental** and **Door.**

### Probe Temperature Readings  
The Environmental Monitor (EM22) I setup in my car can wirelessly monitor probe temperature. Though I was primarily chaffeuring my friends across the state, the Samsara EM-series monitors are ideal for ensuring food safety during transportation, monitoring warehouse conditions, and other applications that rely on environmental data to ensure quality or maintain compliance.

In [309]:
# GET SENSORS HISTORY: probeTemp
# list of sensor ID, field pairs to query
series = [{"widgetId": 212014918118242, "field": "probeTemperature"}]

# HistoryParam | Group ID, time range and resolution, and list of sensor ID, field pairs to query
history_param = samsara.HistoryParam(18750, 1535995860000, 1536042360000, 60000, series, "withPrevious") # fill missing values with previous value

try: 
    # /sensors/history
    probe_response = api_instance.get_sensors_history(access_token, history_param) # re-use Default API instance
    # print a sample of ten readings
    pprint(probe_response.results[-10:])
except ApiException as e:
    print("Exception when calling DefaultApi->get_sensors_history: %s\n" % e)

[{'series': [20653], 'time_ms': 1536041760000},
 {'series': [20653], 'time_ms': 1536041820000},
 {'series': [20653], 'time_ms': 1536041880000},
 {'series': [20653], 'time_ms': 1536041940000},
 {'series': [20653], 'time_ms': 1536042000000},
 {'series': [20193], 'time_ms': 1536042060000},
 {'series': [20193], 'time_ms': 1536042120000},
 {'series': [20193], 'time_ms': 1536042180000},
 {'series': [20193], 'time_ms': 1536042240000},
 {'series': [20193], 'time_ms': 1536042300000}]


In [254]:
# What sort of data structures are we working with here?
print('{}, {}, {}'.format(type(probe_response), type(probe_response.results), type(probe_response.results[0])))

<class 'samsara.models.sensor_history_response.SensorHistoryResponse'>, <class 'list'>, <class 'samsara.models.sensor_history_response_results.SensorHistoryResponseResults'>


### Ambient Temperature Readings
Currently reported ambient temperature (from the same EM22 sensor) in millidegrees celsius.

In [308]:
# GET SENSORS HISTORY: ambientTemp
# list of sensor ID, field pairs to query
series = [{"widgetId": 212014918118242, "field": "ambientTemperature"}]

# HistoryParam | Group ID, time range and resolution, and list of sensor ID, field pairs to query
history_param = samsara.HistoryParam(18750, 1535995860000, 1536042360000, 60000, series, "withPrevious") # minute resolution

try: 
    # /sensors/history
    ambient_response = api_instance.get_sensors_history(access_token, history_param) # re-use Default API instance
    # print a sample of ten readings
    pprint(ambient_response.results[-10:])
except ApiException as e:
    print("Exception when calling DefaultApi->get_sensors_history: %s\n" % e)

[{'series': [22830], 'time_ms': 1536041760000},
 {'series': [22830], 'time_ms': 1536041820000},
 {'series': [22830], 'time_ms': 1536041880000},
 {'series': [22455], 'time_ms': 1536041940000},
 {'series': [22455], 'time_ms': 1536042000000},
 {'series': [22455], 'time_ms': 1536042060000},
 {'series': [22455], 'time_ms': 1536042120000},
 {'series': [22455], 'time_ms': 1536042180000},
 {'series': [22455], 'time_ms': 1536042240000},
 {'series': [22455], 'time_ms': 1536042300000}]


### Door Monitor
The DM11 Door Monitor is a data-logging sensor that detects when doors open and close for powerful visibility and security. In this next API call, I use a different widgetId to access the DM11 rather than the EM22, and pull data from a field labeled **"doorClosed"**:

In [315]:
# GET SENSORS HISTOY: doorClosed
# list of sensor ID, field pairs to query
series = [{"widgetId": 212014918223924, "field": "doorClosed"}]

# HistoryParam | Group ID, time range and resolution, and list of sensor ID, field pairs to query.
history_param = samsara.HistoryParam(18750, 1535995860000, 1536042360000, 3600000, series, "withPrevious") # hourly resolution

try: 
    # /sensors/history
    door_response = api_instance.get_sensors_history(access_token, history_param)
    pprint(door_response.results)
except ApiException as e:
    print("Exception when calling DefaultApi->get_sensors_history: %s\n" % e)

[{'series': [0], 'time_ms': 1535995860000},
 {'series': [1], 'time_ms': 1535999460000},
 {'series': [1], 'time_ms': 1536003060000},
 {'series': [0], 'time_ms': 1536006660000},
 {'series': [0], 'time_ms': 1536010260000},
 {'series': [0], 'time_ms': 1536013860000},
 {'series': [0], 'time_ms': 1536017460000},
 {'series': [0], 'time_ms': 1536021060000},
 {'series': [0], 'time_ms': 1536024660000},
 {'series': [0], 'time_ms': 1536028260000},
 {'series': [0], 'time_ms': 1536031860000},
 {'series': [0], 'time_ms': 1536035460000},
 {'series': [0], 'time_ms': 1536039060000}]


"doorClosed" is a flag indicating whether the current door (the driver's door in my case) is closed or open. It's a boolean value, so there are only two possible values: 1 for closed and 0 for open. 
  
From the look of this response object, it seems I drove most of the way with my door open! In fact, the **magnetic bar had fallen off of my door** and I didn't notice until the end of the trip. For this reason, I'll **leave these data points out** of my visualizations.

### Data Munging

I want to massage these data into a `pandas` dataframe, which will make visualization a lot easier for the sake of this exercise.  
To get to that stage, though, we'll first do a bit of **data manipulation and management** to get our data structures and units straight.

In [317]:
# create list of timestamps
time_list = []
span = range(len(probe_response.results))

# loop through each reading in the sensor history response object
for i in span:
    # store time in milliseconds from each reading
    result = probe_response.results[i].time_ms
    # convert time from milliseconds to datetime
    s = result / 1000.0
    time_list.append(datetime.datetime.fromtimestamp(s).strftime('%Y-%m-%d %H:%M:%S'))
    
# create list of probe temperature readings
probe_temp_list = []

# loop through each reading in the sensor history response object
for i in span:
    # store raw temperature reading
    result = probe_response.results[i].series
    for r in result:
        # convert millidegree celsius to fahrenheit
        r = (r*.0018)+32
        probe_temp_list.append(f"{r:.2f}") # round temperate to second decimal place
        
# create list of ambient temperature readings
ambient_temp_list = []

# loop through each reading in the sensor history response object
for i in span:
    # store raw temperature reading
    result = ambient_response.results[i].series
    for r in result:
        # convert celsius to farenheit
        r = (r*.0018)+32
        ambient_temp_list.append(f"{r:.2f}") # round temperate to second decimal place

# Python 3 to get list of tuples from the two lists
data_tuples = list(zip(probe_temp_list, ambient_temp_list, time_list))

# print a sample of 10 tuples
data_tuples[-20::2]

[('72.62', '73.09', '2018-09-03 23:06:00'),
 ('72.62', '73.09', '2018-09-03 23:08:00'),
 ('71.86', '73.09', '2018-09-03 23:10:00'),
 ('71.86', '73.09', '2018-09-03 23:12:00'),
 ('70.08', '73.09', '2018-09-03 23:14:00'),
 ('69.18', '73.09', '2018-09-03 23:16:00'),
 ('69.18', '73.09', '2018-09-03 23:18:00'),
 ('69.18', '72.42', '2018-09-03 23:20:00'),
 ('68.35', '72.42', '2018-09-03 23:22:00'),
 ('68.35', '72.42', '2018-09-03 23:24:00')]

### Dataframe  
Now that the data looks a little more intelligible, let's put together that dataframe we've been talking about.

In [338]:
# Convert list of tuples to pandas dataframe
# We can also specify column names with the list of tuples
df = pd.DataFrame(data_tuples, columns=['probeTemp', 'ambientTemp', 'Timestamp'])

# Create a plotly table from dataframe
table = ff.create_table(df[100:1000:50]) # show only a subset of total dataframe in table
offline.iplot(table, filename='Temp-table1')

In [339]:
# line graph

# Create and style line for probeTemp
trace1 = go.Scatter(x=df.Timestamp, y=df.probeTemp, name = 'probeTemp (degrees F)', line = dict(color = ('rgb(205, 12, 24)'),
        width = 4))

# Create and style line for ambientTemp
trace2 = go.Scatter(x=df.Timestamp, y=df.ambientTemp, name = 'ambientTemp (degrees F)', line = dict(color = ('rgb(22, 96, 167)'),
        width = 4))

# Combine lines into a list
data = [trace1, trace2]

# Edit the layout
layout = dict(title = 'Temperature Inside My Car, Labor Day Long Haul 2018',
              xaxis = dict(title = 'Hour'),
              yaxis = dict(title = 'Temperature (degrees F)'),
              )

# Plot the lines and layout
fig = dict(data=data, layout=layout)
offline.iplot(fig, filename='probeTemp-line_chart', image='png')

### Plotting Trips Endpoints
Plotly is now integrated with Mapbox. I'll plot lattitude and longitude data of start/end locations for historic fleet trips (i.e. my drive from SD to SF).

In [333]:
## The get_fleet_trips method returns a set of historical trips data for the specified vehicle in the specified time range.

# TripsParam | Group ID, vehicle ID and time range to query.
trips_param = samsara.TripsParam(18750, 212014918214656, 1535995860000, 1536042360000)

try: 
    # /fleet/trips
    trips_response = api_instance.get_fleet_trips(access_token, trips_param) # re-use Default API instance
except ApiException as e:
    print("Exception when calling FleetApi->get_fleet_trips: %s\n" % e)
    
# create lists of end location coordinates and place names
end_lat = []
end_lon = []
locations_name = []
locations_time = []

# add coordinates, name and time for first start location
end_lat.append(trips_response.trips[0].start_coordinates.latitude)
end_lon.append(trips_response.trips[0].start_coordinates.longitude)
locations_name.append(trips_response.trips[0].start_location)
locations_time.append(trips_response.trips[0].start_ms)

# loop through each reading in fleet trips response object
for i in range(len(trips_response.trips)):
    # add coordinates, name, and time to respective lists
    end_lat.append(trips_response.trips[i].end_coordinates.latitude)
    end_lon.append(trips_response.trips[i].end_coordinates.longitude)
    locations_name.append(trips_response.trips[i].end_location)
    locations_time.append(trips_response.trips[i].end_ms)
    
# Python 3 to get list of tuples from the two lists
data_tuples2 = list(zip(locations_name, end_lat, end_lon))

# Convert list of tuples to pandas dataframe
# We can also specify column names with the list of tuples
df2 = pd.DataFrame(data_tuples2, columns=['Location', 'Latitude', 'Longitude'])

# Create a plotly table from dataframe
table2 = ff.create_table(df2)
py.iplot(table2, filename='endtrip-table2')

In [334]:
# To plot on Mapbox maps with Plotly, you'll need a Mapbox account and a Mapbox Access Token
mapbox_access_token = 'insert_your_token_here'

data2 = [go.Scattermapbox(lat=df2.Latitude, lon=df2.Longitude, mode='markers', marker=dict(
        size=17, color='rgb(255, 0, 0)', opacity=0.7), text=df2.Location, hoverinfo='text'),
        go.Scattermapbox(lat=df2.Latitude, lon=df2.Longitude, mode='markers', marker=dict(size=8,
        color='rgb(242, 177, 172)', opacity=0.7), hoverinfo='none')]

layout2 = go.Layout(title='Fleet Trips Start/End Points', autosize=True, hovermode='closest', showlegend=False,
            mapbox=dict(accesstoken=mapbox_access_token, bearing=0, center=dict(lat=35, lon=-117),
            pitch=0, zoom=4.5, style='light'),)

fig2 = dict(data=data2, layout=layout2)

py.iplot(fig2, filename='fleet_trips-map1')

### Conclusion

